In [ ]:
from scipy.fft import fft, ifft, fftshift, fft2, ifft2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

%matplotlib inline 

In [ ]:
# We need a tool that converts images into grayscale (i.e. all pixel values in [0, 1])
def gray_scale_convert(image):
    rescaled_image = np.zeros((image.shape[0], image.shape[1]))
    rescaled_image[:, :] = image
    rescaled_image -= np.min(rescaled_image)
    rescaled_image /= np.max(rescaled_image)
    return rescaled_image

# Noisy Signals

Just to get a sense of the power of the FFT, we look at the signal 

$$
s(t) = \cos(2t) + \sin(3t) + \epsilon(t),  \epsilon(t) \sim \mathcal{N}(0, 1).
$$

In [ ]:
Npts = int(2**8)
times = np.linspace(0., 20.*np.pi, Npts)
rvals = np.random.randn(Npts)
sfun = lambda t: np.cos(2.*t) + np.sin(3.*t)
svals = sfun(times) + rvals

plt.plot(times, svals, 'k')
#plt.plot(times, sfun(times), 'r--')

In [ ]:
sfreq = fft(svals)
sfreq_plot = np.abs(fftshift(sfreq[:]))/np.sqrt(Npts)
kvals = np.arange(0, int(Npts/2)) * (2.*np.pi)/(20.*np.pi)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ax = axes.ravel()
ax[0].plot(times, svals, 'k')
ax[0].set_xlabel(r"$t$")
ax[0].set_ylabel(r"$s(t)$")

ax[1].plot(kvals, sfreq_plot[int(Npts/2):], 'k')
ax[1].set_xlabel(r"$k$")
ax[1].set_ylabel(r"$|\hat{s}(k)|$");


So now, if we want to denoise, we use the same kind of threshholding to maximum arguments we used for the SVD.  This looks like:

In [ ]:
sfmax = np.max(np.abs(sfreq))
indskp = np.ma.log2(np.abs(sfreq)/sfmax) >= -1
sfilter = np.zeros(sfreq.size, dtype=np.complex128)
sfilter[indskp] = sfreq[indskp]
sfilter_plot = fftshift(np.abs(sfilter))/np.sqrt(Npts)
sdenoised = np.real(ifft(sfilter))

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ax = axes.ravel()
ax[0].plot(times, sdenoised, 'k')
#ax[0].plot(times, sfun(times), 'r--')
ax[0].set_xlabel(r"$t$")
ax[0].set_ylabel(r"$s(t)$")

ax[1].plot(kvals, sfilter_plot[int(Npts/2):], 'k')
ax[1].set_xlabel(r"$k$")
ax[1].set_ylabel(r"$|\hat{s}(k)|$");


# Image Analysis

Now, what happens when we use the 2D FFT on Beans?  

In [ ]:
# Find, load, and convert image from textbook data
# Image is in Lecture_Notes folder 
dog_grayscale = io.imread('beans.jpg', as_gray=True)

dog_freq = fft2(dog_grayscale)
dog_freq_plot = np.log10(fftshift(np.abs(dog_freq)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ax = axes.ravel()

ax[0].imshow(dog_grayscale, cmap=plt.cm.gray)
ax[1].matshow(dog_freq_plot)

plt.tight_layout()

So, there isn't really any clear interpretation of Beans in the FFT plot to the right.  Nevertheless, we can still do useful things with it.  To wit, we can look at, as you might have guessed, amplitude comparisons and only keep large Fourier coefficients.  

In [ ]:
max_freq = np.max(np.abs(dog_freq))
indskp = np.ma.log10(np.abs(dog_freq)/max_freq) >= -3
dog_filter = np.zeros((dog_freq.shape[0], dog_freq.shape[1]), dtype=np.complex128)
dog_filter[indskp] = dog_freq[indskp]
dog_reduced = gray_scale_convert(np.real(ifft2(dog_filter)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ax = axes.ravel()

ax[0].imshow(dog_reduced, cmap=plt.cm.gray)
ax[1].imshow(dog_grayscale, cmap=plt.cm.gray)

plt.tight_layout()

So with almost none of the affiliated Fourier spectra, we get a weirdly good recreation of Beans, but in a very different way than what we did with the SVD.  Let's see how the original and altered images look though through that lense.  

In [ ]:
u, s, vh = np.linalg.svd(dog_grayscale)
ur, sr, vhr = np.linalg.svd(dog_reduced)

In [ ]:
plt.plot(np.ma.log10(s/s[0]), color='k')
plt.plot(np.ma.log10(sr/sr[0]), color='r', ls='--')

So we see our FFT based approach is similar to chopping off dimensions in the SVD, though perhaps not quite as systematic in how we can describe the effect. 

## FFT Theory

For $f(t+T)=f(t)$ with $f(t) \in L_{2}([0, T])$, again, we define the Fourier coefficient $\hat{f}_{n}$ for $n\in \mathbb{Z}$ as 

$$
\hat{f}_{n} = \left<f,\hat{e}_{n}\right> = \frac{1}{\sqrt{T}}\int_{0}^{T}f(t) e^{-2\pi i n t/T} dt.  
$$

With a modicum of theory, we can in turn write

$$
f(t) = \sum_{n=-\infty}^{\infty}\hat{f}_{n}\hat{e}_{n}(t), \hat{e}_{n}(t) = \frac{1}{\sqrt{T}}e^{-2\pi i n t/T}.
$$

We can think of the transition from $f(t)$ to the infinte vector $\left(\hat{f}_{n}\right)_{n=-\infty}^{\infty}$ as the _forward_ transform, and the process of summing the terms $\hat{f}_{n}\hat{e}_{n}(t)$ as the _inverse_ transform.  

Now, as we've examined in lecture and the homework, a central issue for Fourier transforms is the rate of decay of $\hat{f}_{n}$.  As we know, if we take 

$$
f(t) = \left\{\begin{array}{rl} 0 & 0 \leq t \leq \pi \\ 1 & \pi < t < 2\pi \end{array}\right.
$$

then 

$$
\hat{f}_{n} = \left\{\begin{array}{rl} \frac{i}{n}\sqrt{\frac{2}{\pi}} & n ~\mbox{odd} \\ 0 & n ~\mbox{even}\end{array}\right.
$$

So that slow rate of decay combined with the limited number of modes of the FFT makes for a tricky situation.  To see what I mean, let's compute.  

In [ ]:
def step_fun(t):
    if 0 <= t and t <= np.pi:
        return 0
    else:
        return 1
    
def interp_freq(t, freq):
    Npts = freq.size
    hlf = int(Npts/2)
    tpi = 2.*np.pi
    nvals = np.append(np.arange(0, hlf+1), np.arange(-hlf+1, 0))
    phase = np.exp(nvals * 1j * t)
    return np.real(np.sum(freq * phase))/Npts

In [ ]:
Npts = int(2**8)
tvals = np.linspace(0, 2.*np.pi, Npts)
fvals = [step_fun(tval) for tval in tvals]
ffreq = fft(fvals)
ffreq_plot = fftshift(np.abs(ffreq))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ax = axes.ravel()

ax[0].plot(tvals, fvals)
ax[1].plot(ffreq_plot[int(Npts/2):])

plt.tight_layout()

In [ ]:
tvalsfine = np.linspace(0, 2.*np.pi, 2*Npts)
interp_step = [interp_freq(tval, ffreq) for tval in tvalsfine]

In [ ]:
plt.plot(interp_step)